In [5]:
import pygame
import math

pygame.init()

BLACK = (0, 0, 0)
PURPLE = (255, 0, 255)
WHITE = (255, 255, 255)
GREEN = (0, 255, 0)

WIDTH, HEIGHT = 800, 700
GRAVITY = 0.1

my_font = pygame.font.SysFont('dejavuserif', 30)

screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("DKY")

class Player:
    def __init__(self, pos, radius, speed, color):
        self.pos = pos
        self.radius = radius
        self.speed = speed
        self.color = color 
        self.lives = 5
        self.score = 0

    def display(self):
        pygame.draw.circle(screen, self.color, self.pos, self.radius)

    def update(self):
        self.pos[0] += self.speed[0]
        self.pos[1] += self.speed[1]
        self.speed[1] += GRAVITY

        if self.pos[0] > WIDTH or self.pos[0] < 0 or self.pos[1] > HEIGHT or self.pos[1] < 0:
            if self.lives > 1:
                self.lives -= 1
                self.pos = [100, 100]
                self.speed = [0, 0]
                
                
        
class Wall:
    def __init__(self, begin, end, color):
        self.begin = begin
        self.end = end
        self.color = color
        self.m = (self.end[1] - self.begin[1]) / (self.end[0] - self.begin[1])
        self.c = self.begin[1] - self.m * self.begin[0]

    def display(self):
        pygame.draw.line(screen, self.color, self.begin, self.end) 

    def collide(self, player):
        z1 = self.end
        z2 = self.begin
        c = player.pos
        r = player.radius
        z1z2 = (z1[0] - z2[0], z1[1] - z2[1])
        z2c = (z2[0] - c[0], z2[1] - c[1])
        a = z1z2[0] * z1z2[0] + z1z2[1] * z1z2[1]
        b = 2 * (z1z2[0] * z2c[0] + z1z2[1] * z2c[1])
        c = z2c[0] * z2c[0] + z2c[1] * z2c[1] - (r * r)
        det = b * b - 4 * a * c
        if det < 0:
            return
        t1 =  (-b + math.sqrt(det)) / (2 * a)
        t0 =  (-b - math.sqrt(det)) / (2 * a)
        t = (t0 + t1) / 2
        if t > 1 or t < 0:
            return
        player.speed[1] = 0
        player.pos[1] -= 1

class Orb:
    def __init__(self, pos, radius, speed, color):
        self.pos = pos
        self.radius = radius
        self.speed = speed
        self.color = color 
        self.alive = True

    def display(self):
        if self.alive:
            pygame.draw.circle(screen, self.color, self.pos, self.radius)

    def collide(self, player):
        x1 = self.pos
        x2 = player.pos
        d1 = x1[0] - x2[0]
        d2 = x1[1] - x2[1]
        d = d1 * d1 + d2 * d2
        r = player.radius
        if d > r * r:
            return
        self.alive = False

class Level:
    def __init__(self, walls, orbs):
        self.walls = walls
        self.orbs = orbs

   
wall1 = Wall((50, 200), (400, 200), WHITE)
walls = [wall1]
orb1 = Orb([500, 500], 5, [0, 0], GREEN)
orb2 = Orb([300, 500], 5, [0, 0], GREEN) 
orbs = [orb1, orb2] 
level1 = Level(walls, orbs)

wall1 = Wall((50, 200), (400, 200), WHITE)
walls = [wall1]
orb1 = Orb([500, 500], 5, [0, 0], GREEN)
orb2 = Orb([300, 500], 5, [0, 0], GREEN)
orb3 = Orb([200, 600], 5, [0, 0], GREEN)
orb4 = Orb([600, 300], 5, [0, 0], GREEN)
orb5 = Orb([350, 600], 5, [0, 0], GREEN)   
orbs = [orb1, orb2, orb3, orb4, orb5] 
level2 = Level(walls, orbs)

levels = [level1, level2]

def main():
    running = True
    player = Player([100, 100], 10, [0, 0], PURPLE)

    level_number = 0
    level = levels[level_number]
    
    while running:
        screen.fill(BLACK)
        
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False
            if event.type == pygame.KEYDOWN:
                if event.key == pygame.K_LEFT:
                    player.speed[0] -= 2
                if event.key == pygame.K_RIGHT:
                    player.speed[0] += 2
                if event.key == pygame.K_SPACE:
                    player.pos[1] -= 10
                    player.speed[1] -= 2
            if event.type == pygame.KEYUP:
                if event.key == pygame.K_LEFT:
                    player.speed[0] = 0
                if event.key == pygame.K_RIGHT:
                    player.speed[0] = 0   

        for wall in level.walls:
            wall.display()
            wall.collide(player)

        for orb in level.orbs:
            orb.display()
            orb.collide(player)

        player.score = 0
        for orb in level.orbs:
            if orb.alive == False:
                player.score += 1
            if player.score == len(level.orbs):
                level_number += 1
                level = levels[level_number]
            
        lives_text_surface = my_font.render("Lives: " + str(player.lives), False, (255, 0, 0))
        score_text_surface = my_font.render("Score: " + str(player.score), False, (255, 0, 0))
        
        player.update()
        player.display()

        screen.blit(lives_text_surface, (0,0))
        screen.blit(score_text_surface, (0,30))
        pygame.display.update()


if __name__ == "__main__":
    main()
    pygame.quit()